# Set up agent for querying structured data

Use the following cell below to run an example of synthetic data that we've generated via the Databricks Assistant.

In [0]:
%python
from pyspark.sql.functions import rand, expr, monotonically_increasing_id
import pandas as pd

# Get parameters
catalog_name = dbutils.widgets.get("catalog")
schema_name = dbutils.widgets.get("schema")
cust_table_name = dbutils.widgets.get("cust_table")
order_table_name = dbutils.widgets.get("order_table")

# Create synthetic customers DataFrame
customers_pdf = pd.DataFrame({
    "customerID": range(1, 35),
    "first_name": [f"First{i}" for i in range(1, 35)],
    "last_name": [f"Last{i}" for i in range(1, 35)],
    "email_address": [f"user{i}@outlook.com" for i in range(1, 35)],
    "gender": ["M" if i % 2 == 0 else "F" for i in range(1, 35)]
})
customers_df = spark.createDataFrame(customers_pdf)

# Create synthetic orders DataFrame
orders_pdf = pd.DataFrame({
    "orderID": range(1, 201),
    "customerID": [((i-1)%100)+1 for i in range(1, 201)],
    "order_date": pd.date_range("2025-01-01", periods=200, freq="D"),
    "order_amount": [round(20 + 80 * (i % 10) * 0.1, 2) for i in range(1, 201)]
})
orders_df = spark.createDataFrame(orders_pdf)

# Set Unity Catalog context
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

# Write to Delta table
customers_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(cust_table_name)
orders_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(order_table_name)

To make this dataset more interesting, we have also manually created Geospatial data for each of the customer addresses.

In [0]:
import pandas as pd

addr_table_name = dbutils.widgets.get("addr_table")

masked_value = "***"

coords = [
    (-37.88349, 145.02783),
    (-37.93171, 145.06929),
    (-37.85207, 145.06217),
    (-37.91003, 145.1446),
    (-37.88458, 145.10201),
    (-37.85044, 145.13842),
    (-37.88187, 145.13429),
    (-37.85478, 145.10957),
    (-37.88728, 145.05943),
    (-37.92583, 145.04561),
    (-37.90407, 145.10888),
    (-37.96014, 145.14423),
    (-37.93331, 145.10819),
    (-37.851, 145.03672),
    (-37.86427, 145.0405),
    (-37.90334, 145.02167),
    (-37.8965, 145.04874),
    (-37.90165, 145.07037),
    (-37.87484, 145.08617),
    (-37.86129, 145.07965),
    (-37.86156, 145.09854),
    (-37.86265, 145.10369),
    (-37.86075, 145.12086),
    (-37.88052, 145.12086),
    (-37.86075, 145.14662),
    (-37.91588, 145.15371),
    (-37.91196, 145.07861),
    (-37.91963, 145.11626),
    (-37.92114, 145.12773),
    (-37.9382, 145.14833),
    (-37.89164, 145.15692),
    (-37.8678, 145.1528),
    (-37.86482, 145.13872),
    (-37.86889, 145.1394)
]
df_coords = pd.DataFrame(coords, columns=["latitude", "longitude"])

# Create synthetic addresses DataFrame
addresses_pdf = pd.DataFrame({
    "addressID": range(1, 35),
    "customerID": range(1, 35),
    "address": masked_value,
    "city": masked_value,
    "state": "VIC",
    "country": "Australia",
    "postal_zip_code": masked_value
})

addresses_pdf_with_coords = pd.concat([addresses_pdf, df_coords], axis=1)

addresses_pdf_with_coords = spark.createDataFrame(addresses_pdf_with_coords)
addresses_pdf_with_coords.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable(addr_table_name)

Using SQL, we've then used COMMENT ON and ALTER TABLE statements respectively to generate table and column comments as context for Genie.

In [0]:
%sql
COMMENT ON TABLE IDENTIFIER(:cust_table) IS "Customer";
COMMENT ON TABLE IDENTIFIER(:addr_table) IS "Customer Address";
COMMENT ON TABLE IDENTIFIER(:order_table) IS "Customer Orders";

In [0]:
%sql
ALTER TABLE IDENTIFIER(:cust_table)
  ALTER COLUMN
    customerID COMMENT 'Unique customer identifier',
    first_name COMMENT 'Customer first name',
    last_name COMMENT 'Customer last name',
    email_address COMMENT 'Customer email address',
    gender COMMENT 'Customer gender';

ALTER TABLE IDENTIFIER(:addr_table)
  ALTER COLUMN
    addressID COMMENT 'Unique address identifier',
    customerID COMMENT 'Customer identifier for address',
    address COMMENT 'Street address',
    city COMMENT 'City of address',
    state COMMENT 'State of address',
    country COMMENT 'Country of address',
    postal_zip_code COMMENT 'Postal or zip code';

ALTER TABLE IDENTIFIER(:order_table)
  ALTER COLUMN
    orderID COMMENT 'Unique order identifier',
    customerID COMMENT 'Customer identifier for order',
    order_date COMMENT 'Date of order',
    order_amount COMMENT 'Order amount in USD';


We can also assign governed tags to columns too. These are useful from an AI governance perspective, to verify if our agentic respects all the governance boundaries through the rules and policies we've set using Attribute Based Access Control.

- Create a [governed tag](https://docs.databricks.com/aws/en/admin/governed-tags/manage-governed-tags).

- Create [ABAC policy](https://docs.databricks.com/aws/en/data-governance/unity-catalog/abac/policies?language=SQL).

An illustrative example has been provided below:

In [0]:
%sql
-- Create UDF with row filter logic

CREATE OR REPLACE FUNCTION row_filter_female_gender(gender STRING)
  RETURNS BOOLEAN
  DETERMINISTIC
  COMMENT 'ABAC utility: Show rows only for female gender'
  RETURN gender IN ('F');

CREATE OR REPLACE FUNCTION row_filter_male_gender(gender STRING)
  RETURNS BOOLEAN
  DETERMINISTIC
  COMMENT 'ABAC utility: Show rows only for male gender'
  RETURN gender IN ('M');

In [0]:
%python
# Create ABAC policy containing who to apply the policy to and how

spark.sql(
    f"""
    CREATE OR REPLACE POLICY `filter_by_female_gender`
    ON TABLE {catalog_name}.{schema_name}.{cust_table_name}
    ROW FILTER row_filter_female_gender
    TO `derek.huang@databricks.com`
    FOR TABLES
    MATCH COLUMNS hasTagValue('gender', 'F') AS gender  -- Trigger when gender' key has 'F' value
    USING COLUMNS (gender); -- Bind tag to the row filter UDF
  """
)

spark.sql(
    f"""
    CREATE OR REPLACE POLICY `filter_by_male_gender`
    ON TABLE {catalog_name}.{schema_name}.{cust_table_name}
    ROW FILTER row_filter_male_gender
    TO `derek.huang@databricks.com`
    FOR TABLES
    MATCH COLUMNS hasTagValue('gender', 'M') AS gender  -- Trigger when gender' key has 'F' value
    USING COLUMNS (gender); -- Bind tag to the row filter UDF
  """
)

In [0]:
%python
# Only set tag if tag does not exist
res = spark.sql(
    f"""
        SELECT column_name, tag_name, tag_value
        FROM information_schema.column_tags
        WHERE catalog_name = '{catalog_name}'
        AND schema_name = '{schema_name}'
        AND table_name = '{cust_table_name}';
    """
)
if res.filter(res.column_name == "gender").count() < 1:
    spark.sql(
        f"""
            SET TAG ON COLUMN {cust_table_name}.gender `gender` = `F`;
        """
    )

# Show results
res = spark.sql(
    f"""
        SELECT column_name, tag_name, tag_value
        FROM information_schema.column_tags
        WHERE catalog_name = '{catalog_name}'
        AND schema_name = '{schema_name}'
        AND table_name = '{cust_table_name}';
    """
)
display(res.filter(res.column_name == "gender"))

# You can make subsequent changes via the UI.

From here, we can now set up Genie spaces to create a seamless experience for our business users.